# Project: Personalized Real Estate Agent
In this project we will build a personalized real estate agent that will help users find the best property for them. The agent will ask the user a series of questions and based on the answers, it will recommend the best property for them. The agent will also provide a list of properties that match the user's preferences.

## Step 1: Set up the environment
Install the required libraries.

In [ ]:
!pip install lancedb
!pip install langchain
!pip install pydantic
!pip install openapi
!pip install python-dotenv
!pip install -U langchain-openai
!pip install gradio

## Step 2: Generating Real Estate Listings
Import the OpenAI key and define the OpenAI model that will be used to generate the real estate listings.

In [1]:
# import the openai key using dotenv
from dotenv import load_dotenv
import os
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

# define the openai model
model_name = "gpt-3.5-turbo"

Generate a list of real estate listings with the following structure:
```markdown
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
```


In [2]:
# Load the model
from openai import OpenAI
client = OpenAI(api_key=openai_key)

import json

# function to generate a listing of at least 10 real estate listings
def generate_listings(model_name=model_name, number_of_listings=10):
    """
    Generate a real esatate listing using the OpenAI language model.

    input:
        llm: OpenAI language model
        number_of_lsitings: number of listings to generate

    returns:
        listings: list of generated real estate listings
    """    
    prompt = \
    """
    You are a real estate agent. Generate a realistic real estate listings with the following information: neighborhood, price, number of bedrooms, number of bathrooms, square footage, brief description of the property, and a brief description of the neighborhood.

    Return the following information for each listing as a JSON object:
    {
        "neighborhood": "string",
        "price": "string",
        "bedrooms": "string",
        "bathrooms": "string",
        "square_footage": "string",
        "description": "string",
        "neighborhood_description": "string"
    }

    Here is an example of a listing:
    {
        "neighborhood": "Green Oaks",
        "price": "$800,000",
        "bedrooms": "3",
        "bathrooms": "2",
        "square_footage": "2,000 sqft",
        "description": "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.",
        "neighborhood_description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
    }
    """
    
    listings = []
    for i in range(number_of_listings):
        completion = client.chat.completions.create(
            model=model_name,
            temperature = 1.8,
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        listings.append(json.loads(completion.choices[0].message.content))
    return listings

# generate 10 real estate listings
listings = generate_listings(model_name=model_name, number_of_listings=10)

Convert the generated real estate listings into a pandas dataframe and save it as a CSV file.

In [3]:
import pandas as pd
listings = pd.DataFrame(listings)
listings.to_csv('listings.csv', index=False)
listings

,neighborhood,price,bedrooms,bathrooms,square_footage,description,neighborhood_description
0,Maplewood,"$500,000",4,3,"2,500 sqft",Don't miss the opportunity to own this stunnin...,"Situated close to Downtown, Maplewood offers a..."
1,Pacific Heights,"$2,500,000",4,3,"3,500 sqft",Experience luxurious living in this exquisite ...,Pacific Heights is a renowned neighborhood kno...
2,Willow Creek,"$650,000",4,3,"2,500 sqft",Discover this exquisite 4-bedroom 3-bathroom h...,Located in the heart of the Willow Creek neigh...
3,Sunny Brooke,"$600,000",4,3,"2,500 sqft","Don't miss out on this spacious 4-bedroom, 3-b...","Sunny Brooke is known for its scenic parks, hi..."
4,Maplewood Terrace,"$500,000",4,3,"2,500 sqft",Welcome to this spacious family home located i...,Maplewood Terrace is known for its top-rated s...
5,Willowbrook,"$650,000",4,3,"2,500 sqft",Situated in the prestigious neighborhood of Wi...,Willowbrook is a highly sought-after neighborh...
6,Willow Creek,"$650,000",4,3,"2,500 sqft","This beautiful 4-bedroom, 3-bathroom home is l...",Willow Creek is a tranquil and scenic neighbor...
7,Sunnyvale Estates,"$950,000",4,3,"2,500 sqft",Welcome to this spacious and elegantly designe...,Sunnyvale Estates is known for its upscale res...
8,Hillside Heights,"$600,000",4,3,"2,500 sqft","Immaculately maintained 4-bedroom, 3-bathroom ...",Hillside Heights is a family-friendly neighbor...
9,Sunnyvale,"$1,200,000",4,3,"2,500 sqft",Located in the sunny neighborhood of Sunnyvale...,Sunnyvale is known for its thriving tech commu...


## Step 3: Storing Listings in a Vector Database
Convert the real estate listings into embeddings and store them in a lancedb vector database.

In [5]:
# settings for the database and csv file
DB_PATH = "homematch_db"
CSV_FILE = "listings.csv"

# create embeddings for the listings using openai text-embedding-ada-002 model
def get_embedding(text, model='text-embedding-ada-002'):
    """
    Get the embedding of a text using the OpenAI text-embedding-ada-002 model.

    input:
        text: text to embed
        model: OpenAI language model

    returns:
        embedding: embedding of the text
    """
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# create a database of the listings
import lancedb
from lancedb.pydantic import LanceModel, vector
def init_db(db_path=DB_PATH, table_name="listings"):
    """
    Initialize the database with the listings table.

    input:
        db_path: path to the database file

    returns:
        conn: connection to the database
        table: listings table
    """
    conn = lancedb.connect(db_path)

    # schema class
    class Listings(LanceModel):
        neighborhood: str
        price: str
        bedrooms: float
        bathrooms: float
        square_footage: str
        description: str
        neighborhood_description: str
        embedding: vector(1536)

    # create the table
    table = conn.create_table(table_name, schema=Listings, mode="overwrite")
    return conn, table

In [6]:
# load the listings into a pandas dataframe
df = pd.read_csv(CSV_FILE)

# combine the descriptions into a single description for embedding
df['combined'] = df['description'] + ' ' + df['neighborhood_description']

# embed the listings into the dataframe
df['embedding'] = df.combined.apply(lambda x: get_embedding(x))
df.head()

,neighborhood,price,bedrooms,bathrooms,square_footage,description,neighborhood_description,combined,embedding
0,Maplewood,"$500,000",4,3,"2,500 sqft",Don't miss the opportunity to own this stunnin...,"Situated close to Downtown, Maplewood offers a...",Don't miss the opportunity to own this stunnin...,"[0.0061960602179169655, 0.01052367128431797, -..."
1,Pacific Heights,"$2,500,000",4,3,"3,500 sqft",Experience luxurious living in this exquisite ...,Pacific Heights is a renowned neighborhood kno...,Experience luxurious living in this exquisite ...,"[0.026304852217435837, 0.015957245603203773, 0..."
2,Willow Creek,"$650,000",4,3,"2,500 sqft",Discover this exquisite 4-bedroom 3-bathroom h...,Located in the heart of the Willow Creek neigh...,Discover this exquisite 4-bedroom 3-bathroom h...,"[0.012823300436139107, 0.004350644536316395, 0..."
3,Sunny Brooke,"$600,000",4,3,"2,500 sqft","Don't miss out on this spacious 4-bedroom, 3-b...","Sunny Brooke is known for its scenic parks, hi...","Don't miss out on this spacious 4-bedroom, 3-b...","[0.015510909259319305, 0.0044397651217877865, ..."
4,Maplewood Terrace,"$500,000",4,3,"2,500 sqft",Welcome to this spacious family home located i...,Maplewood Terrace is known for its top-rated s...,Welcome to this spacious family home located i...,"[0.016096459701657295, 0.009132015518844128, -..."


In [7]:
# create the database and table
db,table = init_db(DB_PATH)

# insert the listings into the database
table.add(df)

## Step 4: Building the User Preference Interface
Build a user interface that asks the user a series of questions to determine their preferences for a property.  In this case we will generate a random user preference using the openAI model.

In [8]:
def buyer_preference(model_name=model_name):
    """
    Generate a buyer preference using the OpenAI language model.

    returns:
        preference: buyer preference
    """
    prompt = \
    """
    You are a home buyer looking for your dream home.

    Answer the following questions:

    Questions:
    What kind of neighborhood are you interested in?
    What is your budget?
    How many bedrooms do you need?
    How many bathrooms do you need?
    What is the minimum square footage you require?
    What is your preferred style of home?
    Do you have any specific requirements for the property?
    What amenities are important to you?

    Keep your answers concise and to the point.  Example answers are provided for reference.

    Answers:
    I am interested in a family-friendly neighborhood with good schools and parks.
    My budget is $450,000.
    I need at least 3 bedrooms.
    I need at least 2 bathrooms.
    I require a minimum of 1,500 square feet.
    I prefer a modern style home with an open floor plan.
    I need a home office and a backyard.
    Amenities that are important to me include a pool and a gym.

    Generate a set of new and unique answers as a single paragraph.
    """
    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return completion.choices[0].message.content

# generate a buyer preference
buyer_pref = buyer_preference(model_name=model_name)
buyer_pref

'I am interested in a community with a strong sense of community and access to outdoor recreational activities. My budget is $600,000. I require at least 4 bedrooms and 3 bathrooms. The minimum square footage I need is 2,000. I prefer a craftsman-style home with a cozy fireplace and a large kitchen. I have a specific requirement for a garage for two cars. Amenities that are important to me include a home theater and a backyard deck for entertaining.'

## Step 5: Searching Based on Preferences
Retrieve the user's preferences from the vector database and find the most similar real estate listings based on the user's preferences.

In [9]:
def search_listing(buyer_pref, table, num_listings=5):
    """
    Search for listings that match the buyer preference.

    input:
        buyer_pref: buyer preference
        table: listings table
        num_listings: number of listings to return

    returns:
        results: list of listings that match the buyer preference
    """
    # get the embedding of the buyer preference
    pref_embedding = get_embedding(buyer_pref)

    # search for similar listings
    results = table.search(pref_embedding).limit(num_listings).to_pandas() 
    return results

# search for listings that match the buyer preference
results = search_listing(buyer_pref, table, num_listings=5)
results

,neighborhood,price,bedrooms,bathrooms,square_footage,description,neighborhood_description,embedding,_distance
0,Sunny Brooke,"$600,000",4.0,3.0,"2,500 sqft","Don't miss out on this spacious 4-bedroom, 3-b...","Sunny Brooke is known for its scenic parks, hi...","[0.015510909, 0.004439765, 0.005454093, -0.024...",0.364213
1,Sunnyvale,"$1,200,000",4.0,3.0,"2,500 sqft",Located in the sunny neighborhood of Sunnyvale...,Sunnyvale is known for its thriving tech commu...,"[0.021225711, 0.0062106047, 0.00248328, -0.006...",0.371885
2,Willow Creek,"$650,000",4.0,3.0,"2,500 sqft","This beautiful 4-bedroom, 3-bathroom home is l...",Willow Creek is a tranquil and scenic neighbor...,"[0.010297609, 0.020173293, 0.004436806, 0.0010...",0.376804
3,Willow Creek,"$650,000",4.0,3.0,"2,500 sqft",Discover this exquisite 4-bedroom 3-bathroom h...,Located in the heart of the Willow Creek neigh...,"[0.0128233, 0.0043506445, 0.001787641, -0.0006...",0.381488
4,Hillside Heights,"$600,000",4.0,3.0,"2,500 sqft","Immaculately maintained 4-bedroom, 3-bathroom ...",Hillside Heights is a family-friendly neighbor...,"[0.017259812, 0.03574297, -0.0042850343, -0.02...",0.389210


### Step 6: Personalizing Listing Descriptions
For each of the recommended real estate listings, generate a personalized description based on the user's preferences.  Maintain factual integrity in the description while emphasizing the features that align with the user's preferences.

In [10]:
def personalized_listing(buyer_pref, listings):
    """
    Generate a personalized listing for the buyer based on the buyer preference and the listings.

    input:
        buyer_pref: buyer preference
        listings: dataframe of listings

    returns:
        personalized_listing: personalized listing for the buyer
    """

    # generate a prompt
    prompt = \
    """
    You are a real estate agent. Generate a personalized real estate listing for a home buyer based on the buyer preference below.

    Buyer Preference:

    {preferences}

    This following is the property description with the neighborhood description and the price.
    Make sure to personalize the listing for the buyer that mathches the buyer preferences as close as possible.
    Do not change factual information, including neighborhood, price, number of bedrooms, number of bathrooms, and square footage.
    If you include price, make sure you use the listing price below, and only include it if it is less than the buyer's budget if provided.
    
    Keep the listing concise and to the point.


    Property Description:

    {description}

    Neighborhood Description:

    {neighborhood_description}

    Listing Price:
    
    {price}

    Personalized Listing:
    """

    # loop over each listing and gnerate a personalized listing
    personalized_listing = []
    for i in range(len(listings)):
        listing = listings.iloc[i]

        # create the context
        context = prompt.format(preferences=buyer_pref, 
                                description=listing['description'], 
                                neighborhood_description=listing['neighborhood_description'],
                                price=listing['price'])

        # generate the personalized listing
        completion = client.chat.completions.create(
            model=model_name,
            temperature=1.5,
            messages=[
                {
                    "role": "user",
                    "content": context
                }
            ]
        )
        personalized_listing.append(completion.choices[0].message.content)
    return personalized_listing

# generate personalized listings for the buyer
personalized_listings = personalized_listing(buyer_pref, results)
    

In [11]:
print(f"Buyer Preference:")
print(buyer_pref)
print("\n")
for i in range(len(personalized_listings)):
    print(f"Personalized Listing {i+1}:")
    print(personalized_listings[i])
    print("\n")

Buyer Preference:
I am interested in a community with a strong sense of community and access to outdoor recreational activities. My budget is $600,000. I require at least 4 bedrooms and 3 bathrooms. The minimum square footage I need is 2,000. I prefer a craftsman-style home with a cozy fireplace and a large kitchen. I have a specific requirement for a garage for two cars. Amenities that are important to me include a home theater and a backyard deck for entertaining.


Personalized Listing 1:
Property Description:

Don't miss out on this beautiful 4-bedroom, 3-bathroom craftsman-style home in the Sunny Brooke neighborhood. The cozy fireplace, large kitchen with granite countertops, and spacious layout make it perfect for entertaining. The home also features a two-car garage, a home theater, and a backyard deck for outdoor gatherings. 

Neighborhood Description:

Sunny Brooke offers a strong sense of community with scenic parks, hiking trails, and top-rated schools nearby. Enjoy the peac

## Step 7: Create an APP Interface using Gradio
Create an app interface using Gradio that allows the user to input their preferences and view the recommended real estate listings with personalized descriptions.

In [12]:
import gradio as gr

# load in the listings
listings = pd.read_csv('listings.csv')

# supporting functions
# generate a new buyer preference
def generate_buyer_pref():
    buyer_pref = buyer_preference(model_name=model_name)
    return buyer_pref

def submit_pref(buyer_pref, num):
    matches = search_listing(buyer_pref, table, num_listings=num)
    personalized_matches = personalized_listing(buyer_pref, matches)
    newstr = ""
    for i in range(len(personalized_matches)):
        newstr+= f"Personalized Listing {i+1}:\n" + personalized_matches[i] + "\n" + ' '
    matches.drop(columns=['embedding','_distance'], inplace=True)
    return matches, newstr

def close_app():
    print('User selected close app.  Shutting down')
    app.close()

with gr.Blocks() as app:
    # display the buyer preference
    with gr.Row():
        # display the buyer preference
        buyer_pref_textbox = gr.Textbox(value="", label="Buyer Preference", 
                                        placeholder="Select Generate Buyer Preference", 
                                        lines=3, show_label=True)
    # display controls
    with gr.Row():
        pref_button = gr.Button("Generate Buyer Preference")
        num = gr.Slider(minimum=1, maximum=10, step=1, label="Number of Listings")
        submit_button = gr.Button("Show Matching Listings")
    # show the matching listings
    with gr.Row():
        matching_listings = gr.DataFrame(label="Showing  Matching Listings", show_label=True)
    # show the agent recommendation
    with gr.Row():
        custom_listings = gr.Textbox(value="", label="Agent Suggested Listings")
   # show all listings
    with gr.Row():
        all_listings = gr.DataFrame(listings, label="All Available Listings", show_label=True)
    # close button
    with gr.Row():
        close_button = gr.Button('Close App')

    # manage app state    
    pref_button.click(generate_buyer_pref, outputs=buyer_pref_textbox)
    submit_button.click(submit_pref, inputs=[buyer_pref_textbox, num], outputs=[matching_listings, custom_listings])
    close_button.click(close_app)

# launch the app
app.launch()



Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


User selected close app.  Shutting down
Closing server running on port: 7860


In [ ]:
app.close()